In [1]:
import pandas as pd

In [2]:
#llamar los datos de periodicidad
per_oct = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_OCT_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_nov = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_NOV_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_dic = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_DIC_2022', dtype=str, usecols = ['ID_INE','Periodicidad'])
per_ene = pd.read_excel('FUENTES IPC - PERIODICIDAD.xlsx', sheet_name='PER_ENE_2023', dtype=str, usecols = ['ID_INE','Periodicidad'])

In [3]:
#une las bases de datos y escribe un nuevo archivo
archivos = [per_oct,per_nov,per_dic,per_ene]
periodicidad = pd.concat(archivos,ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [4]:
print(periodicidad)
print(periodicidad.info())

       index Periodicidad          ID_INE
0          0           01    01.01.01.17.
1          1           01    01.01.01.35.
2          2           03    01.01.01.48.
3          3           01    01.01.01.50.
4          4           03    01.01.01.60.
...      ...          ...             ...
13558  30401           05  08.17.03.2141.
13559  30442           05  08.17.03.2553.
13560  30443           01  08.17.03.2554.
13561  30518           03  08.17.03.9151.
13562  30519           03  08.17.03.9152.

[13563 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13563 entries, 0 to 13562
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         13563 non-null  int64 
 1   Periodicidad  13563 non-null  object
 2   ID_INE        13563 non-null  object
dtypes: int64(1), object(2)
memory usage: 318.0+ KB
None


In [5]:
#Georeferenciacion de octubre 2022 a enero 2023
geo_oct = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-OCT-2022',dtype=str, usecols = ["ID_INE"]) 
geo_nov = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-NOV-2022',dtype=str, usecols = ["ID_INE"]) 
geo_dic = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO-DIC-2022',dtype=str, usecols = ["ID_INE"]) 
geo_en  = pd.read_excel('FUENTES IPC - GEO.xlsx', sheet_name = 'GEO_ENE_2023',dtype=str, usecols = ["ID_INE"]) 

In [6]:
geo_oct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3923 entries, 0 to 3922
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID_INE  3923 non-null   object
dtypes: object(1)
memory usage: 30.8+ KB


In [7]:
# Lista de georeferenciadas
georef = [geo_oct,geo_nov,geo_dic,geo_en]

In [8]:
geo_new = pd.concat(georef, ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [9]:
geo_new

,index,ID_INE
0,0,01.01.01.5622.
1,1,01.01.01.171.
2,2,01.01.01.376.
3,3,01.01.01.5697.
4,4,01.01.01.8991.
...,...,...
6759,7148,06.10.01.10007.
6760,7149,06.10.01.10009.
6761,7151,06.10.01.10032.
6762,7152,03.18.01.10042.


In [11]:
# Fuentes asignadas de octubre 2022 a enero 2023
fuentes_oct = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-OCT-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"]) 
fuentes_nov = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-NOV-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  
fuentes_dic = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-DIC-2022',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  
fuentes_en  = pd.read_excel('FUENTES IPC - ASIGNADAS.xlsx', sheet_name = 'FUENTES-ENE-2023',dtype=str, usecols=["RegCod","DepCod","MunCod","FntCod","NOMBRE_FUENTE","DIRECCION","CANTIDAD_ARTICULOS"])  

In [12]:
#Lista de fuentes asignadas 
fuentes = [fuentes_oct, fuentes_nov, fuentes_dic, fuentes_en]

In [16]:
# Agregar ID_INE en fuentes asignadas

for fuente in fuentes:
    fuente.insert(7,"ID_INE",fuente.RegCod+"."+fuente.DepCod+"."+fuente.MunCod+"."+fuente.FntCod+".")


In [17]:
fuentes_new = pd.concat(fuentes, ignore_index=True).drop_duplicates(subset = ['ID_INE']).reset_index()

In [18]:
fuentes_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12511 entries, 0 to 12510
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               12511 non-null  int64 
 1   RegCod              12510 non-null  object
 2   DepCod              12511 non-null  object
 3   MunCod              12511 non-null  object
 4   FntCod              12511 non-null  object
 5   NOMBRE_FUENTE       12511 non-null  object
 6   DIRECCION           12511 non-null  object
 7   CANTIDAD_ARTICULOS  12511 non-null  object
 8   ID_INE              12510 non-null  object
dtypes: int64(1), object(8)
memory usage: 879.8+ KB


In [19]:
fuentes_new.drop(fuentes_new[fuentes_new.ID_INE.isin(geo_new.ID_INE) ].index, inplace = True)
fuentes_new = fuentes_new.reset_index()

In [20]:
fuentes_new = fuentes_new.merge(periodicidad,how = 'left', on = 'ID_INE')

In [24]:
fuentes_new.drop_duplicates(subset = ["NOMBRE_FUENTE","DIRECCION"],keep="first", inplace=True)

In [26]:
fuentes_new.to_excel('fuentes_faltan.xlsx')

In [ ]:
fuentes_new